In [0]:
!git clone https://github.com/DTU-VAE/VAE.git
%cd /content/VAE
# !git checkout midi-vae
import os
os.environ['PYTHONPATH'] += ":/content/VAE"

import requests

url = "https://storage.googleapis.com/magentadata/datasets/maestro/v2.0.0/maestro-v2.0.0-midi.zip"
zip_file = requests.get(url)
with open("midi.zip", "wb") as zfile:
  zfile.write(zip_file.content)

import zipfile
with zipfile.ZipFile("midi.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/VAE/data/")

In [3]:
!python3 midi/midi.py -h

usage: midi.py [-h] [--epochs N] [--batch-size N] [--sequence-length N]
               [--colab] [--log-interval N] [--bootstrap BOOTSTRAP]

VAE MIDI

optional arguments:
  -h, --help            show this help message and exit
  --epochs N            number of epochs to train (default: 1)
  --batch-size N        input batch size for training (default: 10)
  --sequence-length N   sequence length of input data to LSTM (default: 50)
  --colab               indicates whether script is running on Google Colab
  --log-interval N      how many batches to wait before logging training
                        status
  --bootstrap BOOTSTRAP
                        specifies the path to the model.tar to load the model
                        from


In [0]:
# start training with bootstrapping
!python3 midi/midi.py --colab --bootstrap model_states/model_epoch_1.tar